In #Chialisp, there are many ways to modularize your code. 

https://chialisp.com/examples#modules

Today we are going to talk about `defmacro`. 🧵

In #lisp-based languages (e.g., #clisp #clojure), a macro is a code that generates other code to extend the language. 

#Chialisp also shares a common set of core principles, i.e., treating code as data, and the use of macros by providing `defmacro`.

https://chialisp.com/examples#defmacro

At a glance, `defmacro` looks very similar to how function is defined, i.e., `defun`, and `defun-inline`, but it also has pre-define macro operators that can be evaulauted to expand the code during the compile time. 🤯

https://developers.chia.net/t/why-use-macros-instead-of-functions/297

How can we use macros to extend the #Chialisp language?

Here is very nice `switch` implementation in chialisp by @trepca.

keybase://chat/chia_network.public#chialisp/13357

In addition, being able to generate code by using knowledge at compile time can be useful for creating performant code.

Let's try to create the `sum` function. Since we want it to be reusable and work with a list of any number of items, we make it a recursive function.

Although we just want to add four numbers together, the generated CLVM looks complicated and execution cost is high because the function has to be compiled such that it's able to call itself recursively.

We might tempt to just hard-code to make it perform better, but it means we can't reuse the code anymore.

In this case, we can use `defmacro` to solve performance and reusable issues.

The code looks different, but the generated CLVM is much cleaner and faster. We also can reuse the new macro for any number of items.

The examples show `sum` of 4 and 5 items respectively.

Let's examine another example which is based on the `map` function example from my previous map/reduce/filter tweet. In the implementation, we define a function inline instead of passing it in.

Look at how clean the CLVM generated by a macro comparing to the function one. And the cost is much lower too!

Knowing macro can be helpful for defining a new syntax as well as creating a more performant code. However, as you can see, it is not straightforward to write a code that generates code and can be more difficult to debug.

https://github.com/Chia-Network/clvm_tools/blob/main/README.md#macros

To see more #chialisp macros: 
https://github.com/Chia-Network/clvm_tools/blob/main/clvm_runtime/macros.clvm
https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/utility_macros.clib

Happy Coding #Chialisp!

### `defmacro` in chialisp
### [Macros](https://github.com/Chia-Network/clvm_tools/blob/main/README.md#macros)
> You can also define macros within a module, which act as inline functions. When a previously defined macro operator is encountered, it "rewrites" the existing statement using the macro, passing along the arguments as literals (ie. they are not evaluated).

- [`qq` and `unquote`](../quote/README.md)
- [defmacro, qq, unquote](https://chialisp.com/examples#defmacro)
- [macros.clvm](https://github.com/Chia-Network/clvm_tools/blob/main/clvm_runtime/macros.clvm)
- [utility_macros.clib](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/utility_macros.clib)

## `switch`
### keybase://chat/chia_network.public#chialisp/13357
> I'm learning macros and while they're quite powerful, it's pretty challenging to write, so interested to get any feedback if this could be written better or especially simpler. 
Here's a "switch" statement implemented as a macro:

```clojure
   (defmacro switch code
        (if (r (r code))
            (list if (qq (= (unquote (f code)) (unquote (f (f (r code))))))
                (r (f (r code)))
                (c switch (c (f code) (r (r code))))
            )
            ; last case is default
            (r (f (r code)))
        )
    )

    (defun-inline case-a (amount)
        (* 1000 amount)
    )

    (defun-inline case-b (amount)
        (* 2000 amount)
    )
 
    (switch input
        ("A" case-a 20) 
        ("B" case-b 21) 
        ("default" case-b -1))
```

In #Chialisp, to iterate thru the list, we use recursion.

Compiled function is complicated to support recursion.

But if we know the number of items in advance, we can use macro to generate more performant code.

What is macro?

### `defun` vs `defmacro`
#### sum

In [1]:
clsp = """
(mod items
    (defun sum (items)
        (if (r items)
            (+ (f items) (sum (r items)))
            (f items)
        ) 
    )
    (sum items)
)
"""

In [2]:
sum = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(sum)

solution = Program.to([1, 2, 3, 4])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = sum.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1                                                                                                               
   2 (mod items                                                                                                    
   3     (defun sum (items)                                                                                        
   4         (if (r items)                                                                                         
   5             (+ (f items) (sum (r items)))                                                                     
   6             (f items)                                                                                         
   7         )                                                                                                     
   8     )                                                                                                         
   9     (sum items)                                                                                               
  10 )                                                                                                             
  11                                                                                                               

clvm:

(a (q 2 2 (c 2 (c 3 ()))) (c (q 2 (i 13 (q 16 9 (a 2 (c 2 (c 13 ())))) (q . 9)) 1) 1))                             

solution:

(q 2 3 4)                                                                                                          

cost:

5326

result:

10                                                                                                                 

In [3]:
clsp = """
(mod (v1 v2 v3 v4)
    (defmacro sum items
        (if (r items)
            (list
                + (f items) 
                  (c sum (r items))
            )
            (f items)
        )
    )
    (sum v1 v2 v3 v4)
)
"""

In [4]:
sum = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(sum)

solution = Program.to([1, 2, 3, 4, 5])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = sum.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1                                                                                                               
   2 (mod (v1 v2 v3 v4)                                                                                            
   3     (defmacro sum items                                                                                       
   4         (if (r items)                                                                                         
   5             (list                                                                                             
   6                 + (f items)                                                                                   
   7                   (c sum (r items))                                                                           
   8             )                                                                                                 
   9             (f items)                                                                                         
  10         )                                                                                                     
  11     )                                                                                                         
  12     (sum v1 v2 v3 v4)                                                                                         
  13 )                                                                                                             
  14                                                                                                               

clvm:

(+ 2 (+ 5 (+ 11 23)))                                                                                              

solution:

(q 2 3 4 5)                                                                                                        

cost:

2484

result:

10                                                                                                                 

#### map

In [5]:
clsp = """
(mod (fn . lst)
    (defun map (fn lst)
        (if lst
            (c
                (a fn (f lst))
                (map fn (r lst))
            )
            () 
        )
    )
    (map fn lst)
)
"""
fn_clsp = "(lambda v (* v v))"

map = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(map)

# square the items in the list
print("[bold bright_green]function:")
print_clsp(fn_clsp, line_numbers=False)

fn = Program.to(
    compile_clvm_text(fn_clsp, search_paths=["."])
)

solution = Program.to([fn] + [1, 2, 3, 4, 5])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = map.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1                                                                                                               
   2 (mod (fn . lst)                                                                                               
   3     (defun map (fn lst)                                                                                       
   4         (if lst                                                                                               
   5             (c                                                                                                
   6                 (a fn (f lst))                                                                                
   7                 (map fn (r lst))                                                                              
   8             )                                                                                                 
   9             ()                                                                                                
  10         )                                                                                                     
  11     )                                                                                                         
  12     (map fn lst)                                                                                              
  13 )                                                                                                             
  14                                                                                                               

clvm:

(a (q 2 2 (c 2 (c 5 (c 7 ())))) (c (q 2 (i 11 (q 4 (a 5 19) (a 2 (c 2 (c 5 (c 27 ()))))) ()) 1) 1))                

function:

...

((* 1 1) 1 2 3 4 5)                                                                                                

cost:

11682

result:

(q 4 9 16 25)                                                                                                      

In [8]:
clsp = """
(mod (v1 v2 v3 v4 v5)
    (defmacro map input
        (if (r input)
            (list
                c (qq (fn (unquote (f input))))
                  (c map (r input))
            )
            (qq (c (fn (unquote (f input))) ()))
        )
    )
    (defun-inline fn (v) (* v v))
    (map v1 v2 v3 v4 v5)
)
"""

map = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(map)

solution = Program.to([1, 2, 3, 4, 5])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = map.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1                                                                                                               
   2 (mod (v1 v2 v3 v4 v5)                                                                                         
   3     (defmacro map input                                                                                       
   4         (if (r input)                                                                                         
   5             (list                                                                                             
   6                 c (qq (fn (unquote (f input))))                                                               
   7                   (c map (r input))                                                                           
   8             )                                                                                                 
   9             (qq (c (fn (unquote (f input))) ()))                                                              
  10         )                                                                                                     
  11     )                                                                                                         
  12     (defun-inline fn (v) (* v v))                                                                             
  13     (map v1 v2 v3 v4 v5)                                                                                      
  14 )                                                                                                             
  15                                                                                                               

clvm:

(c (* 2 2) (c (* 5 5) (c (* 11 11) (c (* 23 23) (c (* 47 47) ())))))                                               

solution:

(q 2 3 4 5)                                                                                                        

cost:

5859

result:

(q 4 9 16 25)                                                                                                      